In [8]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import os

In [5]:
loader = DirectoryLoader(r"D:\groq\hybrids\hyb\data",
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
text_chunks = text_splitter.split_documents(documents)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\UMAYARAJ KUMAR\AppData\Local\Temp\ipykernel_44088\3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
vectorstore = Chroma.from_documents(text_chunks, embeddings)

In [12]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})
keyword_retriever = BM25Retriever.from_documents(text_chunks)
keyword_retriever.k =  3

In [13]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.3, 0.7])

In [19]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model = "deepseek-r1:7b")


In [20]:
from langchain.chains import RetrievalQA

normal_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore_retreiver
)

hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ensemble_retriever
)
response = normal_chain.invoke("Who is Harry Potter")

In [21]:
print("Response:", response["result"])

Response: <think>
Okay, so I need to figure out who Harry Potter is based on the context provided. Let me read through the text carefully. 

The first paragraph mentions that "HARRY POTTER and all related characters and elements are trademarks of Warner Bros." That tells me that Harry Potter is a character owned by Warner Bros., probably from a book or movie series.

Looking further down, there's a section titled "Harry Potter And the Sorcerer’s Stone" followed by some information about not being able to reproduce parts of the publication without permission. But this doesn't directly tell me who Harry Potter is; it just mentions a specific book in the series and copyright info.

The helpful answer provided says, "Harry Potter is one of the main characters from J.K. Rowling's 'Harry Potter' book series." That makes sense because I know that J.K. Rowling wrote the original books about Harry and his adventures with other characters like Hermione and Ron.

Wait, but in the context given, t

In [22]:
from langchain.chains import RetrievalQA

normal_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore_retreiver
)

hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ensemble_retriever
)
respons = hybrid_chain.invoke("Who is Harry Potter")

In [23]:
print("Response:", respons["result"])

Response: <think>
Okay, so I need to figure out who Harry Potter is based on the context provided. Let's start by looking at the text snippets given. 

First, there are two identical sections with "THE ROGUE BLUDGER" and details about a conversation between Harry and Dobby. It mentions that someone opened a water jug, and Harry refuses to go home because his friend is Muggle-born and might be first in line if the Chamber has been opened. Dobby tries to reassure him but says Harry risks his own life for his friends.

Next, there are lists of books written by J.K. Rowling, each corresponding to a year at Hogwarts: Year One through Year Seven. These are from "Harry Potter and the..." series.

Then, it mentions that "Harry Potter" is a trademark of Warner Bros., so I shouldn't copy any content without permission.

Looking back at the helpful answer provided earlier, it correctly identifies Harry Potter as the main character. But since I need to simulate thinking through this, let me go ste